In [1]:
import pandas as pd

In [7]:
pd.__version__

'0.22.0'

In [2]:
df_off = pd.read_csv('data/ccf_offline_stage1_train.csv')
df_test = pd.read_csv('data/ccf_offline_stage1_test_revised.csv')

In [3]:
df_off.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN


In [6]:
df_test.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1.0,20160712
1,6949378,1300,3429,30:5,NaN,20160706


In [18]:
coupon_use_count = df_off[df_off.Coupon_id.notnull() & df_off.Date.notnull()].shape[0]
coupon_notuse_count = df_off[df_off.Coupon_id.notnull() & df_off.Date.isnull()].shape[0]
nocoupon_buy_count = df_off[df_off.Coupon_id.isnull() & df_off.Date.notnull()].shape[0]
nocoupon_nobuy_count = df_off[df_off.Coupon_id.isnull() & df_off.Date.isnull()].shape[0]

print('获得优惠卷，并使用：', coupon_use_count)
print('获得优惠卷，没使用：', coupon_notuse_count)
print('没获得优惠卷，消费：', nocoupon_buy_count)
print('没获得优惠卷，没消费：', nocoupon_nobuy_count)

获得优惠卷，并使用： 75382
获得优惠卷，没使用： 977900
没获得优惠卷，消费： 701602
没获得优惠卷，没消费： 0


### 用到的特征

#### 用户特征
- 用户领取优惠券次数
- 用户获得优惠券但没有消费的次数
- 用户获得优惠券并核销次数
- 用户领取优惠券后进行核销率

#### 商家特征
- 商家优惠券被领取次数
- 商家优惠券被领取后不核销次数
- 商家优惠券被领取后核销次数
- 商家优惠券被领取后核销率

#### 优惠卷特征
- 优惠券类型(直接优惠为0, 满减为1)
- 优惠券折率
- 历史出现次数
- 历史核销次数
- 历史核销率

#### 用户与商家交互
- 用户领取商家的优惠券次数
- 用户领取商家的优惠券后不核销次数
- 用户领取商家的优惠券后核销次数
- 用户领取商家的优惠券后核销率

#### 环境特征
- 月份
- 是否是周末

In [167]:
df_user = pd.DataFrame()
df_user['User_id'] = df_off.User_id.unique()
df_user.sort_values(by='User_id', ascending=True, inplace=True)
df_user.index = range(0, df_user.shape[0])
df_user.head(3)

,User_id
0,4
1,35
2,36


#### 用户领取优惠券次数 Coupon_counts

In [168]:
df_off.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN


In [169]:
df_coupon_counts = df_off.groupby('User_id', as_index=False).Coupon_id.count()
df_coupon_counts.columns = ['User_id', 'Coupon_counts']
df_coupon_counts.head(3)

,User_id,Coupon_counts
0,4,2
1,35,4
2,36,2


In [170]:
df_user = pd.merge(df_user, df_coupon_counts, how='outer', on='User_id')
df_user.head(2)

,User_id,Coupon_counts
0,4,2
1,35,4


#### 用户获得优惠券但没有消费的次数 Coupon_nouse_counts

In [171]:
df_coupon_nouse_counts = df_off[df_off.Coupon_id.notnull() & df_off.Date.isnull()].groupby('User_id', as_index=False).Coupon_id.count()
df_coupon_nouse_counts.columns = ['User_id', 'Coupon_nouse_counts']
df_user = pd.merge(df_user, df_coupon_nouse_counts, how='outer', on='User_id')
df_user.head(2)

,User_id,Coupon_counts,Coupon_nouse_counts
0,4,2,2.0
1,35,4,4.0


#### 用户获得优惠券并核销次数 Coupon_use_counts

In [172]:
df_coupon_use_counts = df_off[df_off.Coupon_id.notnull() & df_off.Date.notnull()].groupby('User_id', as_index=False).Coupon_id.count()
df_coupon_use_counts.columns = ['User_id', 'Coupon_use_counts']
df_user = pd.merge(df_user, df_coupon_use_counts, how='outer', on='User_id')
df_user.head(2)

,User_id,Coupon_counts,Coupon_nouse_counts,Coupon_use_counts
0,4,2,2.0,NaN
1,35,4,4.0,NaN


#### 用户领取优惠券后进行核销率 Coupon_use_rate

NameError: name 'df_' is not defined

In [154]:
df_test.groupby('User_id').count()

,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
User_id,,,,,
209,2,2,2,2,2
215,1,1,1,0,1
316,1,1,1,1,1
417,1,1,1,1,1
432,1,1,1,1,1
448,5,5,5,5,5
452,2,2,2,1,2
470,1,1,1,1,1
569,1,1,1,1,1
